In [1]:
import numpy as np

from openforcefield.topology import Molecule, Topology

from openff.system.stubs import ForceField
from openff.system.utils import get_test_file_path

In [2]:
# Load a minimal SMIRNOFF forcefield and Argon topology
argon_ff = ForceField(get_test_file_path('argon.offxml'))

mol = Molecule.from_smiles('[#18]')
mol.generate_conformers(n_conformers=1)

argon_top =  Topology.from_molecules(10 * [mol])

In [3]:
# Use a monkey-patched function to parametrize the topology against a force field
sys_out = argon_ff.create_openff_system(argon_top)

In [4]:
# Look at which ParameterHandler objects from the OpenFF toolkit 
# have been made into Potentialhandler objects
sys_out.handlers.keys()

dict_keys(['vdW'])

In [5]:
# Store this handler to inspect its contents
vdw = sys_out.handlers['vdW']

In [6]:
# Look at some ~metadata
vdw.name, vdw.expression, vdw.independent_variables

('vdW', '4*epsilon*((sigma/r)**12-(sigma/r)**6)', {'r'})

In [7]:
# Return a mapping between atom indices and SMIRKS identifiers
vdw.slot_map

{(0,): '[#18:1]',
 (1,): '[#18:1]',
 (2,): '[#18:1]',
 (3,): '[#18:1]',
 (4,): '[#18:1]',
 (5,): '[#18:1]',
 (6,): '[#18:1]',
 (7,): '[#18:1]',
 (8,): '[#18:1]',
 (9,): '[#18:1]'}

In [8]:
# Return a mapping between SMIRKS identifiers and Potential objects;
# Note the de-duplication, resulting from a many-to-few mapping between
# atoms in the topology and unique parameters in the force field
vdw.potentials

{'[#18:1]': Potential(parameters={'sigma': <Quantity(0.3, 'nanometer')>, 'epsilon': <Quantity(0.1, 'kilojoule / mole')>})}

In [9]:
# Look at this contents of this Potential objects
vdw.potentials['[#18:1]']

Potential(parameters={'sigma': <Quantity(0.3, 'nanometer')>, 'epsilon': <Quantity(0.1, 'kilojoule / mole')>})

In [10]:
# Further, look at the particular value of one of its parameters
vdw.potentials['[#18:1]'].parameters['sigma']

0.3 <Unit('nanometer')>

In [11]:
# Look up, from the highest-level object, this same data, using the
# SMIRKS pattern as a key connecting the topological data to the
# parametrized data
sys_out.handlers['vdW'].potentials[
    sys_out.handlers['vdW'].slot_map[(0,)]
].parameters['sigma']

0.3 <Unit('nanometer')>